In [8]:
import pandas as pd

# URLs for the files
var_names_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv"
data_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv"

# Load column names
cols = pd.read_csv(var_names_url)

# Load data with mixed types warning handling
data = pd.read_csv(data_url, na_values=["9999", "", " ", "Presented but no response", "NA"], low_memory=False)

# Check data types
print(data.dtypes)

# Example of converting a specific column (replace 'column_name' with the actual name)
# data['column_name'] = pd.to_numeric(data['column_name'], errors='coerce')
# Remove completely empty columns
empty = (data.isna().sum() == data.shape[0])
data = data[empty.index[~empty]]  # Keep non-empty columns only

# Display the shape of the DataFrame
print(data.shape)

UNIQUE_id                                        object
UNIQUE_num_records                                int64
ELIGIBLE_consent                                 object
GEO_residence_canada                             object
GEO_province                                     object
                                                 ...   
PSYCH_big_five_inventory_conscientious_score    float64
PSYCH_big_five_inventory_extraverted_score      float64
PSYCH_big_five_inventory_neurotic_score         float64
PSYCH_big_five_inventory_open_score             float64
REMOVE_case                                      object
Length: 1794, dtype: object
(11431, 1779)


In [9]:
print(data[['DEMO_household_income', 'DEMO_age', 'DEMO_gender', 'CONNECTION_social_time_friends_p7d_grouped', 'GEO_province']])

      DEMO_household_income  DEMO_age DEMO_gender  \
0        $20,000 to $24,999      71.0  Non-binary   
1        $70,000 to $79,999      69.0       Woman   
2              Under $5,000      56.0       Woman   
3      $140,000 to $149,999      54.0       Woman   
4        $80,000 to $89,999      30.0         Man   
...                     ...       ...         ...   
11426    $50,000 to $59,999      45.0       Woman   
11427    $80,000 to $89,999      36.0         Man   
11428                   NaN       NaN         NaN   
11429                   NaN       NaN         NaN   
11430    $25,000 to $29,999      31.0       Woman   

      CONNECTION_social_time_friends_p7d_grouped      GEO_province  
0                                5 or more hours  British Columbia  
1                                   1 to 4 hours           Ontario  
2                                5 or more hours            Quebec  
3                                5 or more hours               NaN  
4                 

In [10]:
print(data[['WELLNESS_self_rated_mental_health', 'CONNECTION_social_num_close_friends_grouped', 'CONNECTION_social_time_family_p7d_grouped']])

      WELLNESS_self_rated_mental_health  \
0                                  Poor   
1                             Very good   
2                             Very good   
3                                  Poor   
4                             Very good   
...                                 ...   
11426                         Very good   
11427                         Very good   
11428                              Poor   
11429                              Poor   
11430                              Fair   

      CONNECTION_social_num_close_friends_grouped  \
0                                             NaN   
1                                       5 or more   
2                                       5 or more   
3                                       5 or more   
4                                       5 or more   
...                                           ...   
11426                                         NaN   
11427                                   5 or more   
11428 

In [11]:
# Convert categorical outcome to ordinal numeric values if necessary
category_mapping = {'Poor': 1, 'Fair': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
data['WELLNESS_self_rated_mental_health'] = data['WELLNESS_self_rated_mental_health'].map(category_mapping)

# Verify conversion
print(data['WELLNESS_self_rated_mental_health'].head())

0    1.0
1    NaN
2    NaN
3    1.0
4    NaN
Name: WELLNESS_self_rated_mental_health, dtype: float64


In [12]:
import statsmodels.api as sm
import pandas as pd

# Load variable names and data
var_names_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv"
data_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv"

cols = pd.read_csv(var_names_url)

data = pd.read_csv(data_url, na_values=["9999", "", " ", "Presented but no response", "NA"], low_memory=False)

# Ensure the outcome variable is numeric
category_mapping = {'Poor': 1, 'Fair': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
data['WELLNESS_self_rated_mental_health'] = data['WELLNESS_self_rated_mental_health'].map(category_mapping)

# Ensure categorical predictors are properly treated
X1 = 'CONNECTION_social_time_friends_p7d_grouped'
X2 = 'CONNECTION_social_time_family_p7d_grouped'

# Use OrderedModel for ordinal logistic regression
model = sm.OLS.from_formula(f"WELLNESS_self_rated_mental_health ~ C({X1}) + C({X2})", data=data).fit()

# Show the summary of the model
print(model.summary())


                                    OLS Regression Results                                   
Dep. Variable:     WELLNESS_self_rated_mental_health   R-squared:                       0.046
Model:                                           OLS   Adj. R-squared:                  0.045
Method:                                Least Squares   F-statistic:                     50.24
Date:                               Mon, 25 Nov 2024   Prob (F-statistic):           1.26e-60
Time:                                       15:21:44   Log-Likelihood:                -9781.4
No. Observations:                               6244   AIC:                         1.958e+04
Df Residuals:                                   6237   BIC:                         1.962e+04
Df Model:                                          6                                         
Covariance Type:                           nonrobust                                         
                                                            

In [ ]:
#old
import statsmodels.miscmodels.ordinal_model as ord
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Use OrderedModel for ordinal logistic regression
formula = f"WELLNESS_self_rated_mental_health ~ C({X1}) + C({X2})"
model = ord.OrderedModel.from_formula(formula, data=data, distr='logit')
result = model.fit(method='bfgs')

# Generate predicted probabilities
predicted_probs = result.predict()

# Add predicted probabilities to the data for plotting
data['Prob_Poor'] = predicted_probs[:, 0]  # Probability of "Poor"
data['Prob_Fair'] = predicted_probs[:, 1]  # Probability of "Fair"
data['Prob_Good'] = predicted_probs[:, 2]  # Probability of "Good"
data['Prob_Very_Good'] = predicted_probs[:, 3]  # Probability of "Very Good"
data['Prob_Excellent'] = predicted_probs[:, 4]  # Probability of "Excellent"

# Create a line plot of predicted probabilities for each predictor
sns.set(style="whitegrid")
fig, ax = plt.subplots(figsize=(10, 6))

for category, color in zip(
    ['Prob_Poor', 'Prob_Fair', 'Prob_Good', 'Prob_Very_Good', 'Prob_Excellent'],
    sns.color_palette("viridis", 5)
):
    sns.lineplot(
        x=X1, 
        y=category, 
        data=data, 
        label=category.split("_")[1], 
        ax=ax, 
        color=color
    )

# Add labels and legend
ax.set_title("Predicted Probabilities of Mental Health Categories", fontsize=14)
ax.set_xlabel(f"{X1}", fontsize=12)
ax.set_ylabel("Predicted Probability", fontsize=12)
ax.legend(title="Mental Health Categories", loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Check for empty or missing data
assert X1 in data.columns, f"{X1} is not a valid column in the dataset."
assert not data[X1].isnull().any(), f"{X1} contains missing values."

# Group data if X1 is categorical
if data[X1].dtype == 'object' or data[X1].nunique() < 10:  # Assuming X1 is categorical
    data_grouped = data.groupby(X1).mean().reset_index()
else:
    data_grouped = data

# Plot probabilities
fig, ax = plt.subplots(figsize=(10, 6))
for category, color in zip(
    ['Prob_Poor', 'Prob_Fair', 'Prob_Good', 'Prob_Very_Good', 'Prob_Excellent'],
    sns.color_palette("viridis", 5)
):
    sns.lineplot(
        x=X1,
        y=category,
        data=data_grouped,
        label=category.split("_")[1],
        ax=ax,
        color=color,
    )

# Add labels and legend
ax.set_title("Predicted Probabilities of Mental Health Categories", fontsize=14)
ax.set_xlabel(f"{X1}", fontsize=12)
ax.set_ylabel("Predicted Probability", fontsize=12)
ax.legend(title="Mental Health Categories", loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [13]:
# Standardize case to ensure comparison works regardless of capitalization
data['DEMO_gender'] = data['DEMO_gender'].str.lower()
# Replace any value not 'man' or 'woman' with 'other'
data.loc[~data['DEMO_gender'].isin(['man', 'woman']), 'DEMO_gender'] = 'other'

# Display the selected columns
print(data[['DEMO_household_income', 'DEMO_age', 'DEMO_gender']])

      DEMO_household_income  DEMO_age DEMO_gender
0        $20,000 to $24,999      71.0       other
1        $70,000 to $79,999      69.0       woman
2              Under $5,000      56.0       woman
3      $140,000 to $149,999      54.0       woman
4        $80,000 to $89,999      30.0         man
...                     ...       ...         ...
11426    $50,000 to $59,999      45.0       woman
11427    $80,000 to $89,999      36.0         man
11428                   NaN       NaN       other
11429                   NaN       NaN       other
11430    $25,000 to $29,999      31.0       woman

[11431 rows x 3 columns]


In [ ]:
# Standardize case to ensure comparison works regardless of capitalization
data['DEMO_gender'] = data['DEMO_gender'].str.lower()
# Replace any value not 'man' or 'woman' with 'other'
data.loc[~data['DEMO_gender'].isin(['man', 'woman']), 'DEMO_gender'] = 'other'

# Display the selected columns
print(data[['DEMO_household_income', 'DEMO_age', 'DEMO_gender']])

In [1]:
#updated model
import statsmodels.api as sm
import statsmodels.miscmodels.ordinal_model as ord
import pandas as pd

# Load variable names and data
var_names_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv"
data_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv"

cols = pd.read_csv(var_names_url)
data = pd.read_csv(data_url, na_values=["9999", "", " ", "Presented but no response", "NA"], low_memory=False)

# Map the ordinal outcome variable
category_mapping = {'Poor': 1, 'Fair': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
data['WELLNESS_self_rated_mental_health'] = data['WELLNESS_self_rated_mental_health'].map(category_mapping)

# Ensure categorical predictors are treated properly
X1 = 'CONNECTION_social_time_friends_p7d_grouped'
X2 = 'CONNECTION_social_time_family_p7d_grouped'
X3 = 'DEMO_age'
X4 = 'DEMO_gender'

# Formula for ordinal logistic regression with interaction
formula = (f"WELLNESS_self_rated_mental_health ~ "
           f"C({X1}) + C({X2}) + {X3} * C({X4})")

# Fit the ordinal logistic regression model
model = ord.OrderedModel.from_formula(formula, data=data, distr='logit')
result = model.fit(method='bfgs')

# Show the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 1.243809
         Iterations: 78
         Function evaluations: 82
         Gradient evaluations: 82
                                     OrderedModel Results                                    
Dep. Variable:     WELLNESS_self_rated_mental_health   Log-Likelihood:                -6373.3
Model:                                  OrderedModel   AIC:                         1.278e+04
Method:                           Maximum Likelihood   BIC:                         1.288e+04
Date:                               Wed, 27 Nov 2024                                         
Time:                                       04:48:56                                         
No. Observations:                               5124                                         
Df Residuals:                                   5108                                         
Df Model:                                         13                        

In [15]:
#Old model
import statsmodels.api as sm
import pandas as pd

# Load variable names and data
var_names_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv"
data_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv"

cols = pd.read_csv(var_names_url)

data = pd.read_csv(data_url, na_values=["9999", "", " ", "Presented but no response", "NA"], low_memory=False)

# Ensure the outcome variable is numeric
category_mapping = {'Poor': 1, 'Fair': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
data['WELLNESS_self_rated_mental_health'] = data['WELLNESS_self_rated_mental_health'].map(category_mapping)

# Ensure categorical predictors are properly treated
X1 = 'CONNECTION_social_time_friends_p7d_grouped'
X2 = 'CONNECTION_social_time_family_p7d_grouped'
X3 = 'DEMO_age'
X4 = 'DEMO_gender'

# Use OrderedModel for ordinal logistic regression
model = sm.OLS.from_formula(f"WELLNESS_self_rated_mental_health ~ C({X1}) + C({X2} + X3 + X4)", data=data).fit()

# Show the summary of the model
print(model.summary())

                                    OLS Regression Results                                   
Dep. Variable:     WELLNESS_self_rated_mental_health   R-squared:                       0.046
Model:                                           OLS   Adj. R-squared:                  0.045
Method:                                Least Squares   F-statistic:                     50.24
Date:                               Mon, 25 Nov 2024   Prob (F-statistic):           1.26e-60
Time:                                       15:22:18   Log-Likelihood:                -9781.4
No. Observations:                               6244   AIC:                         1.958e+04
Df Residuals:                                   6237   BIC:                         1.962e+04
Df Model:                                          6                                         
Covariance Type:                           nonrobust                                         
                                                            

In [ ]:
#updated model
import statsmodels.api as sm
import statsmodels.miscmodels.ordinal_model as ord
import pandas as pd

# Load variable names and data
var_names_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv"
data_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv"

cols = pd.read_csv(var_names_url)
data = pd.read_csv(data_url, na_values=["9999", "", " ", "Presented but no response", "NA"], low_memory=False)

# Map the ordinal outcome variable
category_mapping = {'Poor': 1, 'Fair': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
data['WELLNESS_self_rated_mental_health'] = data['WELLNESS_self_rated_mental_health'].map(category_mapping)

# Ensure categorical predictors are treated properly
X1 = 'CONNECTION_social_time_friends_p7d_grouped'
X2 = 'CONNECTION_social_time_family_p7d_grouped'
X3 = 'DEMO_household_income'
X4 = 'GEO_province'

# Formula for ordinal logistic regression with interaction
formula = (f"WELLNESS_self_rated_mental_health ~ "
           f"C({X1}) + C({X2}) + {X3} * C({X4})")

# Fit the ordinal logistic regression model
model = ord.OrderedModel.from_formula(formula, data=data, distr='logit')
result = model.fit(method='bfgs')

# Show the summary of the model
print(result.summary())

In [2]:
#old model
import statsmodels.api as sm
import pandas as pd

# Load variable names and data
var_names_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv"
data_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv"

cols = pd.read_csv(var_names_url)

data = pd.read_csv(data_url, na_values=["9999", "", " ", "Presented but no response", "NA"], low_memory=False)

# Convert the outcome variable to numeric if necessary
category_mapping = {'Poor': 1, 'Fair': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
data['WELLNESS_self_rated_mental_health'] = data['WELLNESS_self_rated_mental_health'].map(category_mapping)

# Define variables
X1 = 'CONNECTION_social_time_friends_p7d_grouped'
X2 = 'CONNECTION_social_time_family_p7d_grouped'
X3 = 'DEMO_household_income'
X4 = 'GEO_province'

# Correct the formula
formula = (f"WELLNESS_self_rated_mental_health ~ "
           f"C({X1}) + C({X2}) + {X3} + C({X4})")

# Fit the model
model = sm.OLS.from_formula(formula, data=data).fit()

# Show the summary of the model
print(model.summary())

                                    OLS Regression Results                                   
Dep. Variable:     WELLNESS_self_rated_mental_health   R-squared:                       0.086
Model:                                           OLS   Adj. R-squared:                  0.074
Method:                                Least Squares   F-statistic:                     7.212
Date:                               Mon, 25 Nov 2024   Prob (F-statistic):           6.75e-40
Time:                                       15:51:30   Log-Likelihood:                -5195.1
No. Observations:                               3329   AIC:                         1.048e+04
Df Residuals:                                   3285   BIC:                         1.075e+04
Df Model:                                         43                                         
Covariance Type:                           nonrobust                                         
                                                            

In [ ]:
import statsmodels.api as sm
import statsmodels.miscmodels.ordinal_model as ord
import pandas as pd

# Load variable names and data
var_names_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv"
data_url = "https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv"

cols = pd.read_csv(var_names_url)
data = pd.read_csv(data_url, na_values=["9999", "", " ", "Presented but no response", "NA"], low_memory=False)

# Map the ordinal outcome variable
category_mapping = {'Poor': 1, 'Fair': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
data['WELLNESS_self_rated_mental_health'] = data['WELLNESS_self_rated_mental_health'].map(category_mapping)

# Check for missing values and remove them
data = data.dropna(subset=['WELLNESS_self_rated_mental_health', 'DEMO_household_income', 'GEO_province'])

# Ensure categorical predictors are treated properly
X1 = 'CONNECTION_social_time_friends_p7d_grouped'
X2 = 'CONNECTION_social_time_family_p7d_grouped'
X3 = 'DEMO_household_income'
X4 = 'GEO_province'

# Formula for ordinal logistic regression with interaction
formula = (f"WELLNESS_self_rated_mental_health ~ "
           f"C({X1}) + C({X2}) + {X3} * C({X4})")

# Debug prints
print("Data loaded and preprocessed.")
print(f"Formula: {formula}")

# Fit the ordinal logistic regression model
model = ord.OrderedModel.from_formula(formula, data=data, distr='logit')
#result = model.fit(method='bfgs', maxiter=500, disp=True)
result = model.fit(method='newton', maxiter=1000, disp=True)


# Show the summary of the model
print(result.summary())

In [ ]:
print(data['GEO_province'].isnull().sum())